In [3]:
import socketserver
import http.server
import urllib
import base64
import pickle
import http
import socket
import select
import requests
import threading
import os

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [5]:
class HTTPServer(socketserver.ThreadingTCPServer):
    
    def __init__(self, addr, request_handler):
        self._thread = None
        self.is_stopped = False
        super().__init__(addr, request_handler)
    
    # from http.server.HTTPServer
    def server_bind(self):
        """Override server_bind to store the server name."""
        socketserver.TCPServer.server_bind(self)
        host, port = self.socket.getsockname()[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
        
    def serve_forever(self, interval=0.05):
        # I attempted to speed up response time by overriding the default 
        # serve_forever with a shorter interval, but it doesn't seem to have worked.
        # oh well. 
        target = super().serve_forever
        self._thread = threading.Thread(target=target, args=(interval,), daemon=True)
        self._thread.start()
        
    def stop(self):
        if self._thread is not None:
            self.shutdown()
            self._thread.join()
            self._thread = None
            self.socket.close()
            self.is_stopped = True
    

In [53]:
def parse_params(params):
    out = {}
    for k, v in params.items():
        if len(v) > 1:
            raise ValueError("Too many arguments for %r: %r" % (k, ", ".join(v)))
        out[k] = v[0]
    return out

def parse_qs(qs):
    return parse_params(urllib.parse.parse_qs(qs))

class HelloProxyError(Exception):
    def __init__(self, code, msg):
        self.response = ProxyResponse(code, msg, {'Connection': 'Close'})
        super().__init__(msg)
        
def get_file_content(fp):
    with open(fp, 'rb') as f:
        return f.read()
        
def content_type_for_file(file):
    ext = os.path.splitext(file)[1]
    if ext == '.json':
        return 'Application/json'
    elif ext == '.css':
        return 'text/css'
    elif ext == '.png':
        return 'image/png'
    elif ext == '.js':
        return 'application/x-javascript'
    elif ext == '.gif':
        return 'image/gif'
    elif ext == '.ico':
        return 'image/icon'
    elif ext == '.html':
        return 'text/html'
    else:
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unrecognized MIME type")
        
def byte_encode(msg):
    if isinstance(msg, str):
        return msg.encode('ascii')
    elif isinstance(msg, bytes):
        return msg
    else:
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Error encoding response data")
        
class ProxyResponse():
    def __init__(self, code, msg, headers=None):
        self.code = code
        self.msg = byte_encode(msg)
        self.headers = headers or {}
        
def proxy_response_for_file(fullpath):
    msg = get_file_content(fullpath)
    
    headers = {
        'Content-Type': content_type_for_file(fullpath),
        'Content-Length': str(len(msg)),
        'Connection': 'Keep-Alive'
    }
    
    return ProxyResponse(200, msg, headers)
    
class HelloProxyHandler(http.server.SimpleHTTPRequestHandler):
    
    def log_message(self, format, *args):
        pass
    
    def handle(self):
        self.close_connection = False
        while not self.close_connection:
            r, w, l = select.select([self.request], [], [], 60*10)
            if self.server.is_stopped:
                break
            if r:
                self.handle_one_request()
            else:
                break
    
    def do_GET(self):
        res = urllib.parse.urlparse(self.path) 
        kw = parse_qs(res.query)
        call = kw.get('call', None)
        try:
            rsp = self._handle_get_internal(res, kw)
        except HelloProxyError as e:
            rsp = e.response
        except Exception as e:
            msg = "???" + str(e.args[0])
            headers = {
                'Connection': 'Close',
                'Content-Type': 'text/plain',
                'Content-Length': str(len(msg))
            }
            rsp = ProxyResponse(http.HTTPStatus.INTERNAL_SERVER_ERROR, msg, headers)
                  
        if rsp.headers['Connection'].lower() == 'close':
            self.close_connection = True
            
        self.send_response(rsp.code)
        for k, v in rsp.headers.items():
            self.send_header(k, v)
        self.end_headers()
        self.wfile.write(rsp.msg)
        self.wfile.flush()
        
    def _handle_get_internal(self, res, kw):

        # reject paths that try to navigate out of 
        # the local directory
        if ".." in res.path:
            raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Path requests cannot contain access local paths via '..'.")
        
        if res.path.startswith('/webservice'):
            _, ws, handler = res.path.split('/',2)
            if handler == 'interface/':
                return self._handle_server_call(res, kw)
            else:
                raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unsupported service path: '%s'"%handler)
        fullpath = os.path.join(self.server.basepath, res.path.lstrip("\\/"))
        if os.path.exists(fullpath):
            return proxy_response_for_file(fullpath)
        else:
            raise HelloProxyError(http.HTTPStatus.NOT_FOUND, "Failed to find file '%s'"%res.path)
        
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Unreachable code path")
        
    def _handle_server_call(self, res, kw):
        url = self.server.proxy_url + self.path
        
        hfwd = dict(self.headers)
        
        # chrome on localhost doesn't seem to handle cookies properly,
        # or I don't understand how it is supposed to work
        # so set the cookie globally for the server
        # This means only one user can use the server at a time,
        # which isn't ideal, but whatever
        
        if self.server.cookie:
            hfwd['Cookie'] = "%s=%s"%(self.server.cookie)
        else:
            hfwd.pop('Cookie', None)
        
        sess = self.server.session
        fwd = sess.get(url, headers=hfwd, verify=False)
        
        msg = fwd.content
        headers = fwd.headers
        
        headers['Content-Length'] = str(len(msg))
        headers.pop('Transfer-Encoding', None)
        
        if 'Set-Cookie' in headers:
            c = headers['Set-Cookie']
            cc = http.cookies.SimpleCookie()
            cc.load(c)
            k,v = cc.popitem()
            self.server.cookie = k, v.value
            
        status_code = fwd.status_code
        
        rsp = ProxyResponse(status_code, msg, headers) 
        return rsp
            
    def _server_call_bad_params(self, call, **kw):
        if kw:
            kws = ", ".join("%s=%s" for i in kw.items())
            kws = ": '%s'" % kws
        else:
            kws = ""
        if call is None:
            call = "No call argument provided"
        else:
            call = '%s'%call
            
        msg = "Bad params for server call: %s%s" % (call, kws)
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, msg)
        
class HelloProxyServer(HTTPServer):
    def __init__(self, localaddr, remoteaddr, basepath):
        self.remoteaddr = remoteaddr
        self.basepath = basepath
        self.proxy_url = "https://%s:%s"%remoteaddr
        self.cookie = None
        self.session = requests.Session()
        self._thread = None
        super().__init__(localaddr, HelloProxyHandler)
        
    def __del__(self):
        self.socket.close()

In [55]:
path = "C:\\Users\\nathan\\Documents\\PBS\\sw test\\hello3.0.8"
serv = HelloProxyServer(('localhost', 12346), ('71.189.82.196', 9), path)
serv.serve_forever()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 64378)
----------------------------------------


Traceback (most recent call last):
  File "c:\program files\python35\lib\socketserver.py", line 625, in process_request_thread
    self.finish_request(request, client_address)
  File "c:\program files\python35\lib\socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\program files\python35\lib\socketserver.py", line 681, in __init__
    self.handle()
  File "<ipython-input-53-10e199584a12>", line 77, in handle
    self.handle_one_request()
  File "c:\program files\python35\lib\http\server.py", line 410, in handle_one_request
    method()
  File "<ipython-input-53-10e199584a12>", line 105, in do_GET
    self.wfile.write(rsp.msg)
  File "c:\program files\python35\lib\socket.py", line 593, in write
    return self._sock.send(b)
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


In [54]:
serv.stop()
del serv

In [ ]:
len(prsp.msg)

In [ ]:
wtf['getUserInfo']

In [37]:
cj = wtf['login'][3]

In [ ]:
import clipboard
clipboard.copy(repr(clipboard.paste()))

In [ ]:
c = '_xsrf=2|b4b9fbb6|460c2f9618a0cff9307336b17df39ca1|1551302762; username-localhost-12346="2|1:0|10:1551822265|24:username-localhost-12346|44:ZWU0MDNlNTg0MWJjNDU3ZWI5M2E1ZGQ2ZDQ2NTY5YTU=|157d230ce506eb1da188dfd4268428ea8860ded16396229e778ee6bba3c94d9f"; username-localhost-12345="2|1:0|10:1553193638|24:username-localhost-12345|44:YzIwYjRmMDE3ODc0NGMyNDhjZWIyYjcxMTZlZTEwZDE=|c635fec3f0b09c8b6326b921e71730635511b3969825dbb805bae73a60b7ac01"; PBSLoggedInState=null'
c = 'a8mheiXN#wu7jcnBecknUw!!=DZ6kAXK0zLa2#8UjdRNtJg!!'
cc = http.cookies.SimpleCookie()
cc.load(c)
cookies = {}
for key, morsel in cc.items():
    cookies[key] = morsel.value
cookies

In [ ]:
serv.session.cookies.set?

In [ ]:
serv.server_cookie_key

In [ ]:
cc.